In [1]:
import io
import re
from functools import partial
from contextlib import redirect_stdout

In [2]:
import utils
inp = utils.get_input(2019, 7)[:-1]
print(inp[:100])

Found cached input for 2019 day 7
3,8,1001,8,10,8,105,1,0,0,21,30,55,80,101,118,199,280,361,442,99999,3,9,101,4,9,9,4,9,99,3,9,101,4,9


## machine

In [3]:
# f(mem: List[int], pointer: int, params: List[int], modes: str, **kwargs)
#     -> Tuple[List[int], int, Optional[str]]

def _math(func, mem, pointer, params, modes, **kwargs):
    a = resolve(mem, params[0], modes[0])
    b = resolve(mem, params[1], modes[1])
    out = params[2]
    res = int(func(a, b))
    _mem = list(mem)
    _mem[out] = res
    return _mem, pointer


def _inp(mem, pointer, params, modes, **kwargs):
    inp = kwargs.get('_input', None)
#     print(f"pop inp: {inp}")
#     if inp is None:
#         inp = input("inp: ")
    if len(inp) == 0:
        return mem, pointer, "paused"
    else:
        inp = inp.pop(0)
    out = params[0]
    _mem = list(mem)
    _mem[out] = int(inp)
    return _mem, pointer


def _out(mem, pointer, params, modes, **kwargs):
    val = resolve(mem, params[0], modes[0])
    print("out:", val)
    return mem, pointer


def _jump(func, mem, pointer, params, modes, **kwargs):
    cond = resolve(mem, params[0], modes[0])
    _pointer = pointer
    if func(cond):
        _pointer = resolve(mem, params[1], modes[1])
    return mem, _pointer


def _halt(mem, pointer, *args, **kwargs):
    return mem, pointer, "halted"

In [4]:
import operator
from functools import partial


def resolve(mem, val, mode):
    return mem[val] if int(mode) == 0 else val


def op(op_func, num_params, mem, pointer, modes, **kwargs):
    params = mem[pointer + 1:pointer + num_params + 1]
    if kwargs.get('_debug', False):
        print("", params)
    p_orig = pointer

    mem, pointer, *state = op_func(mem, pointer, params, modes, **kwargs)

    state = state[0] if state else "running"
    if state != "paused":
        pointer = (pointer + len(params) + 1) if (pointer == p_orig) else pointer
    return mem, pointer, state


def opcode(instruction):
    return int(str(instruction)[-2:])


def modes(instruction):
    return f"{instruction:05d}"[:3][::-1]


def run(ops, mem, pointer, state, **kwargs):
    if state in ["halted", "paused"]:
        return mem, pointer, state

    instruction = mem[pointer]
    _opcode = opcode(instruction)
    _modes = modes(instruction)
    if kwargs.get('_debug', False):
        print(_opcode, _modes, end='')

    _op = ops[_opcode]
    return run(ops, *_op(mem, pointer, _modes, **kwargs), **kwargs)


In [5]:
# partial(_op, op_func, num_params)
OPS = {
    1: partial(op, partial(_math, operator.add), 3),
    2: partial(op, partial(_math, operator.mul), 3),
    3: partial(op, _inp, 1),
    4: partial(op, _out, 1),
    5: partial(op, partial(_jump, lambda x: x != 0), 2),
    6: partial(op, partial(_jump, lambda x: x == 0), 2),
    7: partial(op, partial(_math, operator.lt), 3),
    8: partial(op, partial(_math, operator.eq), 3),
    99: partial(op, _halt, 0)
}

def capture(func, *args, **kwargs):
    f = io.StringIO()
    with redirect_stdout(f):
        ret = func(*args, **kwargs)
    s = f.getvalue()
    return int(re.match("^out: ([0-9]+)", s)[1]), ret   

## part 1

In [ ]:
program = list(map(int, inp.split(',')))
# program = list(map(int, "3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0".split(',')))
outputs = {}


from itertools import permutations
perms = [''.join(p) for p in permutations('01234')]

for p in perms:
    _a = capture(run, OPS, program, 0, "running", _input=[p[0], 0])
    _b = capture(run, OPS, program, 0, "running", _input=[p[1], _a])
    _c = capture(run, OPS, program, 0, "running", _input=[p[2], _b])
    _d = capture(run, OPS, program, 0, "running", _input=[p[3], _c])
    _e = capture(run, OPS, program, 0, "running", _input=[p[4], _d])
    outputs[p] = _e
                    
max_val = max(outputs.values())
print(max_val)
[k for k, v in outputs.items() if v == max_val]


## part 2

In [7]:
# program = list(map(int, "3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5".split(',')))
program = list(map(int, inp.split(',')))
outputs = {}

from itertools import permutations
perms = [''.join(p) for p in permutations('56789')]



for p in perms:
    
    registry = {
        'a': (program, 0),
        'b': (program, 0),
        'c': (program, 0),
        'd': (program, 0),
        'e': (program, 0),
    }
    state = "running"
    
    args = {
        'a': [p[0]],
        'b': [p[1]],
        'c': [p[2]],
        'd': [p[3]],
        'e': [p[4]],
    }
    e_out = 0
    
        
    while state != "halted":

#         for amp in 'abcde':

        a_inp = [args['a'].pop(0), e_out] if len(args['a']) > 0 else [e_out]
        a_out, ret = capture(run, OPS, registry['a'][0], registry['a'][1], "running", _input=a_inp)
        registry['a'] = (ret[0], ret[1])
        state = ret[2]

        b_inp = [args['b'].pop(0), a_out] if len(args['b']) > 0 else [a_out]
        b_out, ret = capture(run, OPS, registry['b'][0], registry['b'][1], "running", _input=b_inp)
        registry['b'] = (ret[0], ret[1])
        state = ret[2]

        c_inp = [args['c'].pop(0), b_out] if len(args['c']) > 0 else [b_out]
        c_out, ret = capture(run, OPS, registry['c'][0], registry['c'][1], "running", _input=c_inp)
        registry['c'] = (ret[0], ret[1])
        state = ret[2]

        d_inp = [args['d'].pop(0), c_out] if len(args['d']) > 0 else [c_out]
        d_out, ret = capture(run, OPS, registry['d'][0], registry['d'][1], "running", _input=d_inp)
        registry['d'] = (ret[0], ret[1])
        state = ret[2]

        e_inp = [args['e'].pop(0), d_out] if len(args['e']) > 0 else [d_out]
        e_out, ret = capture(run, OPS, registry['e'][0], registry['e'][1], "running", _input=e_inp)
        registry['e'] = (ret[0], ret[1])
        state = ret[2]


    outputs[p] = e_out

                    
max_val = max(outputs.values())
print(max_val)
[k for k, v in outputs.items() if v == max_val]

59597414


['97865']